In [2]:
import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
from torch.autograd import Variable
import category_encoders as ce
from sklearn import preprocessing
import numpy as np
import random
import sys
import json
from operator import itemgetter as itg

Dict = []
FrequencyDict = {}
InfrequentDict = []
DataArray = []
TrueDict = {}

with open('AllSets.json') as json_data:
    data = json.load(json_data)

Rtext = []
lasttext = ["Blahblahblah"]
counter = 0
Dict.append("<PAD>")
TrueDict['<PAD>'] = counter
oglasttext = ''
print("Processing Stage 1")
for i, dataz in enumerate(data):
    print(dataz)
    for z, datazz in enumerate(data[dataz]['cards']):
        if 'text' in datazz:
            if oglasttext != datazz['text']:
                lasttext = datazz['text']
                oglasttext = datazz['text']
                lasttext = lasttext.replace("'", "")
                lasttext = lasttext.replace('"', "")
                lasttext = lasttext.replace(")", "")
                lasttext = lasttext.replace("(", "")
                lasttext = lasttext.replace("]", "")
                lasttext = lasttext.replace("[", "")
                lasttext = lasttext.replace("}", "")
                lasttext = lasttext.replace("{", "")
                lasttext = lasttext.replace(".", " <EOS> ")
                for word in lasttext.split():
                    if word not in FrequencyDict:
                        FrequencyDict[word] = 1
                    else:
                        FrequencyDict[word] = FrequencyDict[word] + 1
print("Processing Stage 2")
for i in FrequencyDict:
    if FrequencyDict[i] <= 20:
        if i.split()[0].istitle() == True:
            InfrequentDict.append(i)
print("Processing Stage 3")
for i, dataz in enumerate(data):
    #print(dataz)
    for z, datazz in enumerate(data[dataz]['cards']):
        if 'text' in datazz:
            if oglasttext != datazz['text']:
                lasttext = datazz['text']
                oglasttext = datazz['text']
                lasttext = lasttext.replace("'", "")
                lasttext = lasttext.replace('"', "")
                lasttext = lasttext.replace(")", "")
                lasttext = lasttext.replace("(", "")
                lasttext = lasttext.replace("]", "")
                lasttext = lasttext.replace("[", "")
                lasttext = lasttext.replace("}", "")
                lasttext = lasttext.replace("{", "")
                lasttext = lasttext.replace(".", " <EOS> ")
                for word in lasttext.split():
                    if word in InfrequentDict:
                        lasttext = lasttext.replace(word, " <NAME> ")
                DataArray.append(lasttext)
                for word in lasttext.split():
                    if word not in Dict:
                        Dict.append(word)
                        counter += 1
                        TrueDict[word] = counter
                        
print("Processing Stage 4")
#Dict.append("<EOS>")
MaxLen = 0
MaxStr = []
batch_size = 100
for l in range(len(DataArray)%batch_size):
    DataArray.pop()
for x, i in enumerate(DataArray):
    if len(DataArray[x].split()) > MaxLen:
        MaxLen = len(DataArray[x].split())
        MaxStr = DataArray[x]
print("Processing Stage 5")
BatchTensor = torch.zeros((int(len(DataArray)/batch_size),batch_size, MaxLen), device = 'cuda', dtype=torch.long) 
with torch.no_grad():
    for z in range(int(len(DataArray)/batch_size)):
        for x in range(batch_size):
            for l in range(len(DataArray[z*batch_size+x].split())):
                potato = TrueDict[DataArray[z*batch_size+x].split()[l]]
                BatchTensor[z][x][l] = potato      
            for b in range(MaxLen-l):
                if b != 0:
                    BatchTensor[z][x][l+b] = TrueDict["<PAD>"]

Processing Stage 1
10E
2ED
3ED
4ED
5DN
5ED
6ED
7ED
8ED
9ED
A25
AER
AKH
ALA
ALL
ANA
APC
ARB
ARC
ARN
ATH
ATQ
AVR
BBD
BFZ
BNG
BOK
BRB
BTD
C13
C14
C15
C16
C17
C18
CED
CEI
CHK
CHR
CM1
CM2
CMA
CMD
CN2
CNS
CON
CP1
CP2
CP3
CSP
CST
DD1
DD2
DDC
DDD
DDE
DDF
DDG
DDH
DDI
DDJ
DDK
DDL
DDM
DDN
DDO
DDP
DDQ
DDR
DDS
DDT
DDU
DGM
DIS
DKA
DKM
DOM
DPA
DRB
DRK
DST
DTK
DVD
E01
E02
EMA
EMN
EVE
EVG
EXO
EXP
F01
F02
F03
F04
F05
F06
F07
F08
F09
F10
F11
F12
F13
F14
F15
F16
F17
F18
FBB
FEM
FNM
FRF
FUT
G00
G01
G02
G03
G04
G05
G06
G07
G08
G09
G10
G11
G17
G18
G99
GK1
GK2
GNT
GPT
GRN
GS1
GTC
GVL
H09
H17
HHO
HML
HOP
HOU
HTR
HTR17
ICE
IMA
INV
ISD
ITP
J12
J13
J14
J15
J16
J17
J18
JGP
JOU
JUD
JVC
KLD
KTK
L12
L13
L14
L15
L16
L17
LEA
LEB
LEG
LGN
LRW
M10
M11
M12
M13
M14
M15
M19
MBS
MD1
ME1
ME2
ME3
ME4
MED
MGB
MIR
MM2
MM3
MMA
MMQ
MOR
MP2
MPR
MPS
MRD
NEM
NPH
OARC
OC13
OC14
OC15
OC16
OC17
OC18
OCM1
OCMD
ODY
OE01
OGW
OHOP
OLGC
ONS
OPC2
OPCA
ORI
OVNT
P02
P03
P04
P05
P06
P07
P08
P09
P10
P10E
P11
P15A
P2HG
PAER
PAKH
PAL00
PAL01
PAL02
P

In [4]:

AI = MTGAI(len(TrueDict), 100)
AI = AI.cuda()
LossF1 = nn.CrossEntropyLoss()
OptimF1 = optim.Adam(AI.parameters(), lr=.001)
bestloss = 0
epochs = 10
lengths = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
lenph = []
indexes = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SortedData = torch.zeros((batch_size, MaxLen), device = 'cuda', dtype=torch.long)
firstiter= True
for epoch in range(epochs):
    totalloss = 0
    for i in range(int(len(DataArray)/batch_size)):
        datal = []
        AI.train()
        hidden = AI.init_hidden(batch_size)
        
        
        CardData = BatchTensor[i]
        CardData = CardData.cuda()
        for d in range(batch_size):
            lenph = [g for g in CardData[d] if g!=0]
            lengths[d] = len(lenph)
            indexes[d] = d
        targets = torch.zeros((batch_size, MaxLen), device = 'cuda', dtype=torch.long)
        ### Making the targets
        #print(lengths)
        #print(lengths.sort(reverse=True))
        #lengths, indexes = zip(*sorted(zip(lengths, indexes)))
        #print(lengths)
        #print(indexes)
        #targets = torch.zeros(MaxLen, max(lengths))
        #print(targets)
        for d in range(batch_size):
            #print(d)
            for x in range(lengths[d]-1):
                targets[d][x] = CardData[d][x+1]  
                # Set targets to be the next word in sentence
            #for z in range(MaxLen-lengths[d]-1):
                #targets[d][lengths[d]+z] = 0
                # Pad targets to be equal to padded data
        #for i in range(10):
           # print(CardData[i])    
            #print(targets[i])
        #datal = [(card_dataext, len([g for g in card_dataext if g!=0])) for card_dataext in CardData]
        #datasorted = sorted(datal, key=itg(1) , reverse=True)
        #CDS = torch.zeros((batch_size, MaxLen), device = 'cuda', dtype=torch.long)
        #LDS = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        #for bloopy in range(batch_size):
            #CDS[bloopy] = datasorted[bloopy][0]
            #LDS[bloopy] = datasorted[bloopy][1]
        #lengths.sort(reverse=True)
            #print(lengths)
        #print(CardData[0])
        #print(SortedData[0])
        
        ### Making the targets
            #CardData = (DataArray[i].split())[x]
            #NextData = (DataArray[i].split())[x+1]
            #NextData = torch.tensor(TrueDict[NextData], device='cuda')
            #NextData = NextData.unsqueeze(0)
        OptimF1.zero_grad()
        predictions, hidden = AI(CardData, hidden, batch_size, False)
        #for debug in range(10):
            #print(predictions.view(-1, len(Dict)))
            #print(len(predictions.view(-1, len(Dict))))
            #sys.exit()
            #print(len(predictions))
            #loss = LossF1(torch.narrow(predictions.view(-1, len(Dict)), 0, 0, lengths[x]), torch.narrow(targets[x], 0, 0, lengths[x]))
        loss = LossF1(predictions, targets)

        loss.backward(retain_graph=True)

        totalloss+= loss.item()
                    # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(AI.parameters(), .25)

        OptimF1.step()
        if i != 0:
            print("Trained on ",i*batch_size, " cards.")
            print("Avg loss is: ",totalloss/(i*100))
            
            #total_loss += len(data) * loss.data
    print("Total loss this epoch:", totalloss)
    if bestloss > totalloss or bestloss==0:
        bestloss = totalloss
        print("New best loss! Val: ", bestloss)
        torch.save(AI.state_dict(), "./MTGAIbckup.model.pth")

#print(TrueDict)
print("Done with training")

tensor([[[-0.0368],
         [ 1.7802],
         [-0.0195],
         ...,
         [ 0.0000],
         [ 0.0000],
         [ 0.0000]],

        [[ 1.0150],
         [-0.4068],
         [ 0.9770],
         ...,
         [ 0.0000],
         [ 0.0000],
         [ 0.0000]],

        [[ 0.2954],
         [-2.5393],
         [ 1.1302],
         ...,
         [ 0.0000],
         [ 0.0000],
         [ 0.0000]],

        ...,

        [[ 0.1609],
         [-1.5069],
         [-2.5393],
         ...,
         [ 0.0000],
         [ 0.0000],
         [ 0.0000]],

        [[-0.1565],
         [-2.5393],
         [ 0.2954],
         ...,
         [ 0.0000],
         [ 0.0000],
         [ 0.0000]],

        [[-0.7049],
         [-0.0195],
         [-2.5393],
         ...,
         [ 0.0000],
         [ 0.0000],
         [ 0.0000]]], device='cuda:0', grad_fn=<EmbeddingBackward>)
torch.Size([100, 444, 1])
torch.Size([100, 444, 500])


SystemExit: 

/home/bryce/anaconda3/envs/pt/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
AI = MTGAI(len(TrueDict), 100)
AI = AI.cuda()
AI.load_state_dict(torch.load("./MTGAIbckup.model.pth"))
hidden = AI.init_hidden(1, 1)
temperature = 10
num_words = 10
initword = Dict[random.randint(0, len(Dict))]
print(initword, '', end='')
initword = TrueDict[initword]
initword = torch.tensor(initword, device = 'cuda')

for i in range(num_words):
    #print(i)
    #print(Dict[9374]
    output, hidden = AI(initword, hidden, 1, True)
    word_weights = output.squeeze().data.div(temperature).exp().cpu()
    word_idx = torch.multinomial(word_weights, 1)[0]
    initword.data.fill_(word_idx)
    for searchword, searchid in TrueDict.items():
        if searchid == word_idx:
            word = searchword


    print(word + ' ', end='')
print("Done")

RuntimeError: Error(s) in loading state_dict for MTGAI:
	size mismatch for Embed1.weight: copying a param of torch.Size([3398, 100]) from checkpoint, where the shape is torch.Size([13301, 131]) in current model.
	size mismatch for GRU.weight_ih_l0: copying a param of torch.Size([1500, 100]) from checkpoint, where the shape is torch.Size([1500, 131]) in current model.
	size mismatch for Lin1.weight: copying a param of torch.Size([3398, 500]) from checkpoint, where the shape is torch.Size([13301, 500]) in current model.
	size mismatch for Lin1.bias: copying a param of torch.Size([3398]) from checkpoint, where the shape is torch.Size([13301]) in current model.

In [3]:
class MTGAI(nn.Module):
    
    def __init__(self, Dsize, input_size):
        super(MTGAI, self).__init__()
        self.Embed1 = nn.Embedding(Dsize, 1, padding_idx=0)
        self.GRU = nn.GRU(1, 500, 3, batch_first=True) # input size, hidden size, num of layers to use
        self.Drop1 = nn.Dropout(.5)
        self.Drop2 = nn.Dropout(.5)
        self.Lin1 = nn.Linear(500, Dsize)
        #self.Lin1.bias.data.fill_(0)
        #self.Lin1.weight.data.uniform_(-.1, .1)
        self.input_size = input_size
        self.hidden = 0
        
    def forward(self, x, h, batch_size, test):
        output = []
        if test == False:
            y = self.Embed1(x)
            print(y)
            print(y.size())
            y, newh = self.GRU(y, h)
            print(y.size())
            sys.exit()
            y = self.Lin1(y)
            #for z in x:
                #y = self.Embed1(z).view(1, 1, -1)
                #y, newh = self.GRU(y, h)
                #y = self.Lin1(y)
                #output.append(y)
        if test == True:
            y = x
            y = y.unsqueeze(0)
            #print(y)
            y = self.Embed1(y)
            y = y.unsqueeze(0)
            y, newh = self.GRU(y, h)
            y = self.Lin1(y)
        return y, newh
    
    def init_hidden(self, batch_size, input_size=1):
        weight = next(self.parameters()).data
        return Variable(weight.new(3, 100, 500).zero_())